<a href="https://colab.research.google.com/github/chungntu/1DCNN-LSTM-ResNet/blob/main/Problem_1_Spring_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# ===============================
# Bài 1:Hệ phần tử lò xo
# ===============================

# kết nối giữa các nút tại mỗi phần tử
element_nodes = np.array([
    [1, 2],
    [2, 3],
    [2, 4]
], dtype=int)

number_elements = element_nodes.shape[0]
number_nodes = 4

# Khởi tạo vectơ chuyển vị, vectơ lực và ma trận độ cứng
displacements = np.zeros((number_nodes, 1), dtype=float)
force = np.zeros((number_nodes, 1), dtype=float)
stiffness = np.zeros((number_nodes, number_nodes), dtype=float)

# Tải trọng tác dụng tại nút 2
# Lưu ý Python đánh chỉ số từ 0
force[1, 0] = 10.0

# Ma trận độ cứng phần tử
k_local = np.array([[ 1.0, -1.0],
                    [-1.0,  1.0]])

# Ghép nối ma trận độ cứng tổng thể
for e in range(number_elements):
    dof = element_nodes[e] - 1
    i, j = dof[0], dof[1]
    stiffness[np.ix_([i, j], [i, j])] += k_local

# Điều kiện biên và lời giải
prescribed_dof = np.array([1, 3, 4], dtype=int) - 1

all_dof = np.arange(number_nodes)
active_dof = np.setdiff1d(all_dof, prescribed_dof)

# Giải hệ phương trình
K_ff = stiffness[np.ix_(active_dof, active_dof)]
f_f  = force[active_dof, :]

u_f = np.linalg.solve(K_ff, f_f)

# Vectơ chuyển vị
u = np.zeros((number_nodes, 1))
u[active_dof, :] = u_f

# Tính phản lực
reactions = stiffness @ u - force

# ===== Kết quả đầu ra (tương tự outputDisplacementsReactions) =====
print("K (Ma trận độ cứng tổng thể):\n", stiffness)
print("\nVectơ lực f:\n", force)
print("\nChuyển vị u:\n", u)

print("\nPhản lực (K*u - f):\n", reactions)
print("\nPhản lực tại các bậc tự do bị ràng buộc:")
for nd in (prescribed_dof + 1):  # Quay lại hệ số 1 (1-based) để hiển thị
    print(f"  Nút {nd}: R = {reactions[nd-1,0]:.6g}")

K (Ma trận độ cứng tổng thể):
 [[ 1. -1.  0.  0.]
 [-1.  3. -1. -1.]
 [ 0. -1.  1.  0.]
 [ 0. -1.  0.  1.]]

Vectơ lực f:
 [[ 0.]
 [10.]
 [ 0.]
 [ 0.]]

Chuyển vị u:
 [[0.        ]
 [3.33333333]
 [0.        ]
 [0.        ]]

Phản lực (K*u - f):
 [[-3.33333333]
 [ 0.        ]
 [-3.33333333]
 [-3.33333333]]

Phản lực tại các bậc tự do bị ràng buộc:
  Nút 1: R = -3.33333
  Nút 3: R = -3.33333
  Nút 4: R = -3.33333


# New Section